In [1]:
from google.colab import drive
drive.mount('./drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./drive


Bism  
# Imports

In [0]:
import pandas as pd
import numpy as np
import os
import IPython.display as ipd
import librosa
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.style.use('ggplot')
from tqdm import tqdm_notebook


#SKLEARN
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics as sklm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE

#KERAS
from keras import models
from keras.models import Model, load_model
from keras import callbacks
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


# Implementation 6  (PHONE LEVEL)
## Using L2Training set
## L2 as val+test set
## AUTOENCODER PER PHONE

### Training

#### MODEL

In [0]:
!ls './drive/My Drive/Thesis Work/PhoneLevel_Autoencoder_MFCC_TrainL2_TestL2/'

phone_means   test_labels.npy  test_typeOfError.npy  x_test.npy
phone_models  test_phones.npy  train_phones.npy      x_train.npy


In [0]:
x_trainALL = np.load('./drive/My Drive/Thesis Work/PhoneLevel_Autoencoder_MFCC_TrainL2_TestL2/x_train.npy')
train_phones = np.load('./drive/My Drive/Thesis Work/PhoneLevel_Autoencoder_MFCC_TrainL2_TestL2/train_phones.npy')

In [0]:
train_phones.shape

(65113,)

In [0]:
x_trainALL.shape

(65113, 13, 50, 3)

In [0]:
#commented to avoid accedental runs
'''for phone in set(train_phones):
  np.save("./drive/My Drive/Thesis Work/Implementation6/phone_means/mean_"+phone+".npy",\
          np.max(np.abs(x_trainALL[train_phones == phone]),(0,1,2)))''';

In [0]:
print(set(train_phones))

{'AY', 'AH', 'K', 'TH', 'L', 'EH', 'IY', 'AA', 'AE', 'sil', 'sp', 'ER', 'AW', 'DH', 'OW', 'Z', 'UW', 'N', 'HH', 'AO', 'UH', 'IH', 'EY', 'V', 'S', 'D', 'JH', 'F', 'T', 's', 'P', 'Y', 'M', 'G', 'NG', 'OY', 'W', 'CH', 'ZH', 'B', 'SH', 'R'}


In [0]:
phone = 'L'
train_max = np.load('./drive/My Drive/Thesis Work/PhoneLevel_Autoencoder_MFCC_TrainL2_TestL2/phone_means/mean_'+phone+".npy")
x_train = x_trainALL[train_phones == phone] / train_max

In [0]:
x_train.shape

(2636, 13, 50, 3)

In [0]:
print(np.max(x_train))
print(np.min(x_train))

0.7600943778948096
-1.0


In [0]:
np.nan in x_train

False

In [0]:
'''noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_train_noisy = np.clip(x_train_noisy, -1, 1.)''';

In [0]:
#x_train_noisy.shape

In [0]:
input_img = Input(shape=(13, 50, 3))  # adapt this if using `channels_first` image data format

x = Conv2D(64, (2, 3), activation='tanh', padding='valid')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='tanh', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (3, 12, 32)

x = BatchNormalization()(encoded)
x = Conv2D(128, (3, 3), activation='tanh', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='tanh', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = BatchNormalization()(x)
decoded = Conv2DTranspose(3, (2, 3), activation='tanh', padding='valid')(x)

autoencoder = Model(input_img, decoded)

autoencoder.compile(optimizer= keras.optimizers.Adam(), loss='mean_squared_error');

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 13, 50, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 48, 64)        1216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 24, 64)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 24, 64)         256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 24, 128)        73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 12, 128)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 3, 12, 128)        512       
__________

In [0]:
callbacklist=[callbacks.History(),callbacks.ModelCheckpoint('./jawadmodel.h5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=5)]
autoencoder.fit(x_train, x_train, batch_size=256, epochs=100, callbacks=callbacklist)

Epoch 1/100
2636/2636 [==============================] - 1s 423us/step - loss: 4.5529e-04
Epoch 2/100
2636/2636 [==============================] - 1s 408us/step - loss: 4.8183e-04
Epoch 3/100
2636/2636 [==============================] - 1s 393us/step - loss: 4.5339e-04
Epoch 4/100
2636/2636 [==============================] - 1s 402us/step - loss: 4.7317e-04
Epoch 5/100
2636/2636 [==============================] - 1s 395us/step - loss: 5.0182e-04

Epoch 00005: loss improved from inf to 0.00050, saving model to ./jawadmodel.h5
Epoch 6/100
2636/2636 [==============================] - 1s 393us/step - loss: 4.0902e-04
Epoch 7/100
2636/2636 [==============================] - 1s 392us/step - loss: 3.9429e-04
Epoch 8/100
2636/2636 [==============================] - 1s 387us/step - loss: 3.8985e-04
Epoch 9/100
2636/2636 [==============================] - 1s 386us/step - loss: 5.9057e-04
Epoch 10/100
2636/2636 [==============================] - 1s 386us/step - loss: 6.8567e-04

Epoch 00010: loss

In [0]:
modelLoss = _.history['loss'][-1]
modelLoss

0.00039253752102385724

In [0]:
phone

'L'

In [0]:
!cp ./jawadmodel.h5 "./drive/My Drive/Thesis Work/PhoneLevel_Autoencoder_MFCC_TrainL2_TestL2/phone_models/"$phone".h5"

### TESTING

In [0]:
!ls './drive/My Drive/Thesis Work/PhoneLevel_Autoencoder_MFCC_TrainL2_TestL2'

phone_means   test_labels.npy  test_typeOfError.npy  x_test.npy
phone_models  test_phones.npy  train_phones.npy      x_train.npy


In [0]:
x_testALL = np.load('./drive/My Drive/Thesis Work/PhoneLevel_Autoencoder_MFCC_TrainL2_TestL2/x_test.npy')
test_phones = np.load('./drive/My Drive/Thesis Work/PhoneLevel_Autoencoder_MFCC_TrainL2_TestL2/test_phones.npy')
test_labelsALL = np.load('./drive/My Drive/Thesis Work/PhoneLevel_Autoencoder_MFCC_TrainL2_TestL2/test_labels.npy')

In [0]:
phone

'L'

In [0]:
#MAX normalization
train_max = np.load('./drive/My Drive/Thesis Work/PhoneLevel_Autoencoder_MFCC_TrainL2_TestL2/phone_means/mean_'+phone+".npy")
x_test = x_testALL[test_phones == phone] / train_max
test_labels = test_labelsALL[test_phones == phone]

In [0]:
x_test.shape

(1012, 13, 50, 3)

In [0]:
test_labels.shape

(1012,)

In [0]:
#x_test = np.vstack((x_test[test_labels == 1][:sum(test_labels == 0)],x_test[test_labels == 0])) #have to permute this also

In [0]:
#test_labels = np.vstack((test_labels[test_labels == 1][:sum(test_labels == 0)],test_labels[test_labels == 0])) #have to permute this also

In [0]:
#test_labels = test_labels.reshape(-1)

In [0]:
#shuffledIndexes = np.random.permutation(range(x_test.shape[0]))
#x_test = x_test[shuffledIndexes]
#test_labels = test_labels[shuffledIndexes]

In [0]:
'''noise_factor = 0.5
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 
x_test_noisy = np.clip(x_test_noisy, -1, 1)''';

In [0]:
autoencoder = load_model('./drive/My Drive/Thesis Work/PhoneLevel_Autoencoder_MFCC_TrainL2_TestL2/phone_models/'+phone+'.h5') #load previous one without noise

In [0]:
autoencoder.evaluate(x_test[test_labels == 1],x_test[test_labels == 1],batch_size=1)

912/912 [==============================] - 3s 3ms/step


0.0004416479345341111

In [0]:
autoencoder.evaluate(x_test[test_labels == 0],x_test[test_labels == 0])

100/100 [==============================] - 0s 902us/step


0.00046363551286049187

In [0]:
x_test.shape

(1012, 13, 50, 3)

In [0]:
validationSetLength = 150 #initial some from test as validation set
accs = []
f1mis = []

for threshold in np.arange(0.0000,0.00100,0.00001):
  predictions = [1 if autoencoder.evaluate(x_test[i:i+1],x_test[i:i+1],verbose=0)<=threshold\
   else 0 for i in range(validationSetLength)]
  
  accs.append(sklm.accuracy_score(test_labels[:validationSetLength],predictions))
  f1mis.append(sklm.precision_recall_fscore_support(test_labels[:validationSetLength],predictions)[2][0])

print("Max Achievable Accuracy (Overall): ", max(accs))
print("Max Achievable Accuracy (Overall) on Threshold: ", np.argmax(accs))

print("Max Achievable F-1 score Threshold: ", np.argmax(f1mis))
print("Max Achievable Accuracy On This Threshold: ", accs[np.argmax(f1mis)])
print("Max Achievable F-1 score for Mispronunciations: ", max(f1mis))


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Max Achievable Accuracy (Overall):  0.8
Max Achievable Accuracy (Overall) on Threshold:  89
Max Achievable F-1 score Threshold:  25
Max Achievable Accuracy On This Threshold:  0.19333333333333333
Max Achievable F-1 score for Mispronunciations:  0.3163841807909605


In [0]:
0.0000+0.00001*25

0.00025

In [0]:
validationSetLength = 150 #initial some from test as validation set
threshold = 0.00025

predictions = [1 if autoencoder.evaluate(x_test[i:i+1],x_test[i:i+1],verbose=0)<=threshold\
 else 0 for i in range(validationSetLength,x_test.shape[0])] 

In [0]:
print('Accuracy = ', sklm.accuracy_score(test_labels[validationSetLength:],predictions))

Accuracy =  0.09164733178654293


In [0]:
y_test = test_labels[validationSetLength:]
y_pred = predictions

In [0]:
print("Phone: ",phone)
print(classification_report(y_test, y_pred, target_names=['Mispronounced','Correct']))
print(confusion_matrix(y_test, y_pred, labels=range(2)))

print(sklm.roc_curve(y_test,y_pred))
print('FAR = ', sklm.roc_curve(y_test,y_pred)[0][1]*100,'%')
print('FRR = ', (1 - sklm.roc_curve(y_test,y_pred)[1][1])*100,'%')
print("AUC_SCORE = ", sklm.roc_auc_score(y_test,y_pred)*100,'%')

Phone:  L
               precision    recall  f1-score   support

Mispronounced       0.08      1.00      0.16        72
      Correct       1.00      0.01      0.02       790

    micro avg       0.09      0.09      0.09       862
    macro avg       0.54      0.50      0.09       862
 weighted avg       0.92      0.09      0.03       862

[[ 72   0]
 [783   7]]
(array([0., 0., 1.]), array([0.        , 0.00886076, 1.        ]), array([2, 1, 0]))
FAR =  0.0 %
FRR =  99.1139240506329 %
AUC_SCORE =  50.44303797468355 %
